1. Longest Increasing Subsequence
2. Increasing Triplet Subsequence
3. Russian Envelopes

---

# 1. Longest Increasing Subsequence

The goal is to find the length of the LIS.

*You have already solved this before.* 

Maintain an array which tracks elements -- your job is to maintain this array in an increasing manner.

Given `3, 5, 1, 7`, your array will evolve like so:
* `[3]`
* `[3, 5]` because `5` is greater than the last element
* `[1, 5]` -- this is tricky:
    * `1` is clearly smaller than both, but you donot want to discard the current sequence, because `7` appears next.
    * Instead, you `swap` the number in the right place -- this works because ANY number smaller than 3, could have been in 3's place, and the size of the LIS would not change.
* `[1, 5, 7]` -- again, this is NOT the correct subsequence, but the size is correct.

Now, say you actually wanted to track the Longest Subsequence, and not just the size?

# 2. Increasing Triplet Subsequence

An extension of LIS -- does the array contain three indices `i, j, k` such that `array[i] < array[j] < array[k]`.

In [1]:
class IncreasingTriplet:
    def solve(self, nums):
        triplet = []
        for ix, n in enumerate(nums):
            if ix == 0:
                triplet.append(n)
            else:
                if n > triplet[-1]:
                    triplet.append(n)
                else:
                    # It can only have two elements, of which you need to replace
                    # the first, or the 2nd.
                    if n > triplet[0]:
                        triplet[1] = n
                    else:
                        triplet[0] = n
                    
            if len(triplet) == 3:
                return True
            
        return False

In [5]:
o = IncreasingTriplet()
print(o.solve([1,2,3,4,5]))
print(o.solve([5, 4, 3, 2, 1]))
print(o.solve([2,1,5,0,4,6]))

True
False
True


# 3. Russian Envelopes

You have an array of envelopes `(w, h)`. Can you find how many envelopes can be enclosed sequentially? An envelope can be enclosed in another if both the height and width are smaller.

*This seemed like a LIS question, where you could just sort by size and then see which ones fit the next.*

But this approach is not straightforward.

Let's try a more basic graph-oriented approach:
1. Generate an index graph that marks which indices can be *enveloped* into another.
2. Run a topological sort and compute the longest path using DFS.

In [13]:
from functools import lru_cache

class RussianEnvelopeDFS:
    def solve(self, envs):
        graph = {}
        for i in range(len(envs)):
            for j in range(len(envs)):
                if i != j:
                    if envs[i][0] < envs[j][0] and envs[i][1] < envs[j][1]:
                        if i in graph:
                            graph[i].append(j)
                        else:
                            graph[i] = [j]
                            
        @lru_cache(None)
        def dfs(n):
            if n in graph:
                d = 0
                for x in graph[n]:
                    d = max(d, 1 + dfs(x))
                return d
            else:
                return 1
            
        return max([dfs(i) for i in range(len(envs))])

In [16]:
o = RussianEnvelopeDFS()
print(o.solve([[5,4],[6,4],[6,7],[2,3]]))
print(o.solve([[1,1],[1,1]]))

3
1


Unfortunately, this works -- but has a $n^2$ solution.

The $nlogn$ solution is a modified LIS -- you sort the envelopes, increasing order width and *decreasing* order height. This way, when you run LIS on the heights, no two envelops of the same width will be clubbed together. 

Also, this way, LIS on the heights will give you the right solution.

In [10]:
graph = {
    5: [11],
    7: [11, 8],
    3: [8, 10],
    11: [2, 9, 10],
    8: [9],
    9: [10]
}

from functools import lru_cache

@lru_cache(None)
def dfs(n):
    if n in graph:
        d = 0
        for x in graph[n]:
            d = max(d, 1 + dfs(x))
        return d
    else:
        return 1
    
for n in [5, 11, 2, 7, 8, 9, 3, 10]:
    print("(%d) %d" % (n, dfs(n)))

(5) 4
(11) 3
(2) 1
(7) 4
(8) 3
(9) 2
(3) 4
(10) 1


# 4. Maximum Length of Pair Chain

Given pairs `[a, b]` where `a < b`, find the longest chain of pairs `[a b], [c d]` such that `b < c`.

```
[1, 2] [2, 3] [3, 4]


[1, 2] [3, 2] [3, 4]
2, 2, 4


VALID
[1, 2] [3, 5] [6, 7] ==> [1, 2] [5, 3] [7, 6]

INVALID
[1, 2] [1, 3] [4, 5] ==> [1, 2] [3, 1] [4, 5]
```

One solution is to exploit the LIS algo. By definition, for a valid LIS, `b` must be less than `c` AND `d`.

So you first sort the pairs, then you reverse every second pair, and run LIS on the 1st index numbers.

In [27]:
class MaxLenPairLIS:
    def solve(self, pairs):
        pairs.sort()
        print(pairs)
        for i, p in enumerate(pairs):
            if i%2 == 0:
                continue
            p[0], p[1] = p[1], p[0]
            
        lis = [pairs[0][1]]
        for i in range(1, len(pairs)):
            v = pairs[i][1]
            if v > lis[-1]:
                lis.append(v)
            else:
                start = 0
                end = len(lis)-1
                while start <= end:
                    mid = start + (end - start)//2
                    if lis[mid] < v:
                        start = mid+1
                    else:
                        end = mid - 1
                        
                lis[start] = v
                
        return len(lis)

In [28]:
o = MaxLenPairLIS()
print(o.solve([[1,2],[2,3],[3,4]]))
print(o.solve([[1,2],[7,8],[4,5]]))
print(o.solve([[-10,-8],[8,9],[-5,0],[6,10],[-6,-4],[1,7],[9,10],[-4,7]]))

[[1, 2], [2, 3], [3, 4]]
2
[[1, 2], [4, 5], [7, 8]]
3
[[-10, -8], [-6, -4], [-5, 0], [-4, 7], [1, 7], [6, 10], [8, 9], [9, 10]]
5


This does not return the right solution.

The other approach is to construct a graph, and run DFS for longest path using memo.

In [29]:
from functools import lru_cache

class MaxLenPairDFS:
    def solve(self, pairs):
        graph = {}
        for i in range(len(pairs)):
            for j in range(len(pairs)):
                if i != j:
                    if pairs[i][1] < pairs[j][0]:
                        if i in graph:
                            graph[i].append(j)
                        else:
                            graph[i] = [j]
                            
        @lru_cache(None)
        def dfs(i):
            if i not in graph:
                return 1
            else:
                d = 0
                for n in graph[i]:
                    d = max(d, 1 + dfs(n))
                return d
            
        return max([dfs(x) for x in graph.keys()] + [1])

In [30]:
o = MaxLenPairDFS()
print(o.solve([[1,2],[2,3],[3,4]]))
print(o.solve([[1,2],[7,8],[4,5]]))
print(o.solve([[-10,-8],[8,9],[-5,0],[6,10],[-6,-4],[1,7],[9,10],[-4,7]]))

2
3
4
